#### Importando as bibliotecas

In [4]:
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time
import copy
from unicodedata import normalize

#### Criando as funções necessárias para acessar as árvores TRIE

In [5]:
class TrieNode(object):

    
    def __init__(self, char: str):   # __init__ é um método especial para fazer construtores
        self.char = char   # caractere do nodo atual
        self.filhos = []   # nodos filhos
        self.pFinalizada = False   # se é o último nodo e a palavra terminou
        self.indices = []   # lista vazia para nodos que não são término de palavra
        ### não é um índice único e sim uma lista, pois podem existir títulos de música repetidos com mais de um artista


#Definindo função que localiza uma string na arvore
def findString(select, dados, raiz, palavra: str) -> (bool, []):
    """
      1. Se a palavra existe e tem algum índice associado, retorna verdadeiro e a lista de índices dela
      2. Se a palavra não existe, retorna falso e uma lista com sugestões de palavras
      
      Sendo o parâmetro select utilizada para indicar se a busca é por artista ou música (0 música, 1 artista)
    """
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    stringSugerida = ""  # guarda os caracteres já encontrados para sugerir os próximos
    
    if not raiz.filhos:   # se o nodo raiz não tiver nenhum filho, trivial, retorna falso
        return False, []
    for char in palavra:
        charNaoEncontrado = True
        for filho in nodo.filhos:
            if filho.char == char:
                charNaoEncontrado = False   # assinala que o char foi encontrado
                stringSugerida = stringSugerida+char
                nodo = filho   # passa iteração para o nodo filho
                break
        
        if charNaoEncontrado:   # chama a função que procura sugestões e retorna a tupla (False, lista de sugestões)
            return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
    # Caso passe por todos os caracteres sem retornar false, então significa que a palavra foi encontrada
    # Resta saber se aquele nodo é um nodo final com um índice associado
    if nodo.pFinalizada:
        return True, nodo.indices
    else:
        return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
# Função que formata string desconsiderando maiúsculas e minúsculas e que remove acentos
def formataString(string):
    string = normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')
    return string.upper()

def sugereStrings(select, dados, nodo, string, maxSugestoes):   # onde select igual a 1 significa que se procura um artista e False, música
        listaDeSugestoes = []   # cria lista vazia para adicionarmos as sugestões
        procuraSugestoes(nodo, string, listaDeSugestoes)   # chama a função que percorre a trie procurando sugestões
        
        itera = copy.deepcopy(listaDeSugestoes)   # utilizado para iteração na lista que está sendo modificada enquanto o loop itera
        
        # substitui os indices pelos títulos
        for indice in itera:
            listaDeSugestoes.remove(indice)   # remove o índice 
            if select:
                listaDeSugestoes.append(dados[indice]['Artista']) 
            else:
                listaDeSugestoes.append(dados[indice]['Titulo']) 
            
        
        listaDeSugestoes = sorted(listaDeSugestoes)   # ordena a lista alfabeticamente
        return listaDeSugestoes[:maxSugestoes]   # retorna apenas os n elementos da lista definidos por maxSugestoes

    
# função que dado um nodo, uma parte correta de uma string (que existe na TRIE), uma lista de sugestoes, retorna uma lista de sugestões
def procuraSugestoes(nodo, string, listaDeSugestoes):
        
        if nodo.pFinalizada:   
            listaDeSugestoes.append(nodo.indices[0])   # adiciona nova sugestão na lista
        
        for filho in nodo.filhos:            
            procuraSugestoes(filho, string+filho.char, listaDeSugestoes)   # recursão para cada filho 

        return        

#### Fazendo a leitura dos arquivos

In [28]:
#Faz a leitura do arquivo de registros
with (open('database.bin', 'rb')) as openfile:
    database = pickle.load(openfile)
    
#Leitura do arquivo de indices de anos
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

#Leitura do arquivo com a arvore TRIE de titulos de musicas
with open('trieMusicas.bin','rb') as openfile:
    trieMusicas = pickle.load(openfile)
        
#Leitura do arquivo com a arvore TRIE de nomes de artistas
with open('trieArtistas.bin','rb') as openfile:
    trieArtistas = pickle.load(openfile)
    
with (open('topArtistas.bin', 'rb')) as openfile:
    topArtistas = pickle.load(openfile)
        
with (open('topMusicas.bin', 'rb')) as openfile:
    topMusicas = pickle.load(openfile)

## Definição das Querys

#### Query que retorna um numero N de pesquisas em ordem de relevancia, filtrando por artista/ano opcionalmente
###### Opcional: Depois criar um algoritmo que tente achar se tem mais de uma musica do mesmo artista em um ano

In [1]:
def relevancia(dados, top, ano=None, artista=None):    
    #Retornando de um determinado ano
    if(ano != None and artista == None):
        #Consultando do arquivo de indices qual o index inicial do ano que o usuario entrou
        comeco = -1 #Se estiver em -1 eh pq nao encontrou o ano
        for x in indices:
            if x['Ano'] == ano:
                comeco = x['Min']
        if(comeco != -1):
            for i in range(comeco, comeco+top):
                print(dados[i])
        else:
            print('O ano pesquisado não esta na base de dados')
            
    #Retornando de um determinado artista
    if(artista != None and ano == None):
        indicesArtista = findString(1, dados, trieArtistas, artista)[1] #retorna os indices do arquivo principal com registros do artista
        if(type(indicesArtista[0]) == str):
            print('Artista não encontrado')
            print('Sugestões de artistas:')
            print(indicesArtista)
        else:
            listaMusicas2 = []
            for i in indicesArtista:
                listaMusicas2.append(dados[i])

            listaMusicas = copy.deepcopy(listaMusicas2) #Faz uma copia de lista2 para lista, pq lista2 so referencia os registros de dados ao inves de criar eles novamente
        
            for i in range(len(listaMusicas)-1):
                for j in range(i+1, len(listaMusicas)-1):
                    if(listaMusicas[i]['Titulo'] == listaMusicas[j]['Titulo']):
                        listaMusicas[i]['Pontos'] = listaMusicas[i]['Pontos'] + listaMusicas[j]['Pontos']
                        listaMusicas.pop(j)
        
            listaMusicas = sorted(listaMusicas, key = lambda tup: (tup["Pontos"]), reverse=True)
        
            if(len(listaMusicas) < top): #Faz com que a query retorne apenas o numero de entradas pedido na variavel top
                top = len(listaMusicas)
            
            for i in range(top):
                print(listaMusicas[i])    
            
    #Retorna o top geral
    if(ano == None and artista == None):
        for i in range(top):
            print(dados[topMusicas[i][0]])
        
    #Retorna o top dos artistas em determinado ano
    if(artista != None and ano != None):
        #Faz o mesmo procedimento que a busca por artista
        indicesArtista = findString(1, dados, trieArtistas, artista)[1] #retorna os indices do arquivo principal com registros do artista
        if(type(indicesArtista[0]) == str):
            print('Artista não encontrado')
            print('Sugestões de artistas:')
            print(indicesArtista)
        else:
            listaMusicas2 = []
            for i in indicesArtista:
                listaMusicas2.append(dados[i])

            listaMusicas = copy.deepcopy(listaMusicas2) #Faz uma copia de lista2 para lista, pq lista2 so referencia os registros de dados ao inves de criar eles novamente
        
            for i in range(len(listaMusicas)-1):
                for j in range(i+1, len(listaMusicas)-1):
                    if(listaMusicas[i]['Titulo'] == listaMusicas[j]['Titulo']):
                        listaMusicas[i]['Pontos'] = listaMusicas[i]['Pontos'] + listaMusicas[j]['Pontos']
                        listaMusicas.pop(j)
        
            listaMusicas = sorted(listaMusicas, key = lambda tup: (tup["Pontos"]), reverse=True)
        
            listaMusicasAno = [] #Lista com as musicas do artista no ano pedido
        
            for i in range(len(listaMusicas)):
                if(listaMusicas[i]['Ano'] == ano):
                    listaMusicasAno.append(listaMusicas[i])
            
            if(len(listaMusicasAno) < top):
                top = len(listaMusicasAno)
            if(len(listaMusicasAno) > 0):
                for i in range(top):
                    print(listaMusicasAno[i]) 
            else:
                print("Esse artista não teve nenhuma música no top 200 da billboard nesse ano")

#### Testes da funcao de relevancia

In [86]:
#relevancia(database, 10, 2011, 'katy Perry') #especificando artista e ano
#relevancia(database, 5,5000) #especificando apenas ano
#Na query abaixo eh preciso especificar que a variavel que esta sendo passado eh o artista!!
#relevancia(database, 46,artista='Demi Moore') #especificando apenas o artista
#relevancia(database, 20) #sem especificacao nenhuma

#### Query que retorna uma lista de sugestões para artista/música dada um pedaço de string

In [9]:
findString(1, database, trieArtistas, "Daives")[1]

['Da Brat',
 'Daddy Dewdrop',
 'Daddy Yankee',
 'Daft Punk',
 'Dale & Grace',
 'Dale Ward',
 'Damita Jo',
 'Damn Yankees',
 'Dan + Shay',
 'Dan Baird',
 'Dan Fogelberg',
 'Dan Fogelberg/Tim Weisberg',
 'Dan Hartman',
 'Dan Hill',
 'Dan Hill (Duet With Vonda Shepard)']

In [32]:
def retiraFeaturing(artistaString):
    
    if " Featuring" in artistaString:   # Se a música tiver um featuring, deve-se retirá-lo
            indexFeat = artistaString.find(" Featuring")   # pega índice do char space antes de Featuring
            artistaString = artistaString[:indexFeat]   # retorna apenas o primeiro artista
        
    return artistaString   # caso não caia no if, simplesmente retorna o artista assim como veio

#### Query que retorna os N artistas mais relevantes de determinado ano

In [88]:
def relevanciaArtistaAno(dados, indiceFile, ano, maxArtistas):
    achou = 0 #muda para 1 se o ano de entrada estiver no database
    # Loop que pega o ano passado para procurar no arquivo principal o artista mais relevante
    for entrada in range(len(indiceFile)):
        if indiceFile[entrada]['Ano'] == ano:
            index = entrada
            indexMin = indiceFile[index]['Min']
            indexMax = indiceFile[index]['Max']
            achou = 1
    
    if(achou == 0):
        print("O ano inserido não conta na base de dados")
    else:    
        artistasDict = {}
    
        # Itera pelos indices daquele determinado ano construindo um dicionário de artistas e pontos acumulados
        while indexMin <= indexMax:
        
            artistaIterado = retiraFeaturing(dados[indexMin]['Artista'])
            if artistaIterado in artistasDict:
                artistasDict[artistaIterado] += dados[indexMin]['Pontos']
            else:
                artistasDict[artistaIterado] = dados[indexMin]['Pontos']
        
            indexMin += 1
        
        #artistasDict = sorted(artistasDict.items(), key = lambda tup: (tup[1]["Pontos"]), reverse=True)
        dictSorted = sorted(artistasDict.items(), key=lambda kv: kv[1], reverse = True)
    
        return dictSorted[:maxArtistas]


In [90]:
relevanciaArtistaAno(database, indices, 20000, 5)

O ano inserido não conta na base de dados


#### Query que retorna os N artistas mais relevantes no geral

In [29]:
def topArtistasQuery(topArtistas, n, reversa):   # sendo n o número máximo de artistas
    
    if reversa:
        return topArtistas[:-(n+1):-1]
    
    return topArtistas[:n]

In [31]:
topArtistasQuery(5, False)

[('Madonna', 53607),
 ('Rihanna', 49051),
 ('Elton John', 47983),
 ('Mariah Carey', 46615),
 ('Usher', 45357)]

In [19]:
print(topMusicas[1][1])
print(topMusicas[2][1])
print(topMusicas[3][1])

4478
4360
4208
